In [2]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
from scipy.optimize import minimize_scalar

# Configuration parameters
CONFIG = {
    "quad_range": (-5, 5),
    "quad_points": 400,
    "scale": np.sqrt(np.pi)
}

def optimize_homodyne_angle(eta_est, gamma_est, r=10/(10*np.log10(np.e)), avg_photon=5):
    """Calculate optimal homodyne angle using estimated parameters"""
    numerator = (1 - eta_est) + 2*(gamma_est**2)*avg_photon
    denominator = eta_est*(np.exp(2*r) - np.exp(-2*r))
    return np.arctan(np.sqrt(numerator/denominator))

def estimate_noise_params(var_q, var_p, r):
    """Estimate loss (eta) and dephasing (gamma) from quadrature variances"""
    eta = (var_q - 0.5) / (0.5*(np.exp(-2*r) - 1))
    gamma = np.sqrt( (var_p - eta*np.exp(2*r)/2 - (1-eta)/2) / (2*(2*np.sinh(r)**2 + 1)) )
    return np.clip(eta, 0, 1), np.maximum(gamma, 0)

def adaptive_gkp_simulation(
    prep_state: list,
    epsilon: float,
    noise_params: dict,
    num_samples: int = 1000
) -> dict:
    """
    Adaptive GKP simulation with optimized homodyne measurement
    Returns optimized quadrature data and noise estimates
    """
    # Preliminary measurements at 0 and π/2
    quad = np.linspace(*CONFIG["quad_range"], CONFIG["quad_points"]) * CONFIG["scale"]
    r = 10/(10*np.log10(np.e))  # 10dB squeezing
    
    # Initial measurements
    prog = sf.Program(1)
    with prog.context as q:
        GKP(state=prep_state, epsilon=epsilon) | q
        if 'loss' in noise_params:
            LossChannel(noise_params['loss']) | q
        if 'gamma' in noise_params:
            theta = np.random.normal(0, np.sqrt(2*noise_params['gamma']))
            Rgate(theta) | q
        MeasureHomodyne(0) | q

    eng = sf.Engine("bosonic")
    results = eng.run(prog, shots=num_samples)
    var_q = np.var(results.samples)

    prog = sf.Program(1)
    with prog.context as q:
        GKP(state=prep_state, epsilon=epsilon) | q
        if 'loss' in noise_params:
            LossChannel(noise_params['loss']) | q
        if 'gamma' in noise_params:
            theta = np.random.normal(0, np.sqrt(2*noise_params['gamma']))
            Rgate(theta) | q
        MeasureHomodyne(np.pi/2) | q

    results = eng.run(prog, shots=num_samples)
    var_p = np.var(results.samples)

    # Estimate noise parameters
    eta_est, gamma_est = estimate_noise_params(var_q, var_p, r)
    
    # Calculate optimal angle
    phi_opt = optimize_homodyne_angle(eta_est, gamma_est)
    
    # Final optimized measurement
    prog = sf.Program(1)
    with prog.context as q:
        GKP(state=prep_state, epsilon=epsilon) | q
        if 'loss' in noise_params:
            LossChannel(noise_params['loss']) | q
        if 'gamma' in noise_params:
            theta = np.random.normal(0, np.sqrt(2*noise_params['gamma']))
            Rgate(theta) | q
        MeasureHomodyne(phi_opt) | q

    results = eng.run(prog, shots=num_samples)
    
    return {
        'phi_opt': phi_opt,
        'eta_est': eta_est,
        'gamma_est': gamma_est,
        'samples_opt': results.samples,
        'var_q': var_q,
        'var_p': var_p
    }

# Usage example
noise_params = {'loss': 0.2, 'gamma': 0.1}
results = adaptive_gkp_simulation(
    prep_state=[np.pi/2, 0],  # GKP |+> state
    epsilon=0.08,
    noise_params=noise_params,
    num_samples=1000
)

print(f"Optimal angle: {np.rad2deg(results['phi_opt']):.1f}°")
print(f"Estimated loss: {results['eta_est']:.3f}")
print(f"Estimated dephasing: {results['gamma_est']:.3f}")


KeyError: 1